# Практическое задание 4
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО: Ксенофонтов Григорий М05-204а


### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе. 

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils) 


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса. 
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой. 

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно 

## Предобработка данных (2 балла)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/data | cp /content/drive/MyDrive/text2bash_data/test_data.csv /content/data/test.csv | cp /content/drive/MyDrive/text2bash_data/train.csv /content/data/train.csv

Mounted at /content/drive


In [2]:
!git clone -b nlc2cmd https://github.com/IBM/clai/
!pip install sentencepiece transformers

Cloning into 'clai'...
remote: Enumerating objects: 4162, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 4162 (delta 69), reused 207 (delta 46), pack-reused 3900
Receiving objects: 100% (4162/4162), 131.85 MiB | 40.12 MiB/s, done.
Resolving deltas: 100% (2538/2538), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.8 MB/s eta 0:00:00


In [3]:
import sys
PATH_TO_CLAI_UTILS = "/content/clai/utils" ## YOUR CODE HERE ##
sys.path.append(PATH_TO_CLAI_UTILS)

In [4]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Setting bashlex grammar using file: /content/clai/utils/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [5]:
train_data = pd.read_csv('data/train.csv')
train_data.head()

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [6]:
test_data = pd.read_csv('data/test.csv')
test_data.head()

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
import re


def clean_text(text):
    ### YOUR CODE HERE ###
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    
    text = text.lower()
    text = re.sub("[^a-z ]+", "", text)
    words = filter(lambda x: x != '' and x not in stop_words, text.split())
    words = list(map(lambda x: ps.stem(x), words))
    return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [9]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [10]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [11]:
## YOUR CODE HERE ###
with open('text', 'a', encoding='utf-8') as file:
    for text in train_data['text_cleaned']:
        file.write(' '.join(text) + '\n')

with open('cmd', 'a', encoding='utf-8') as file:
    for text in train_data['cmd_cleaned']:
        file.write(text + '\n')

text_trainer = spm.SentencePieceTrainer.train(input='text', model_prefix='text', model_type='bpe', vocab_size=3000)
cmd_trainer = spm.SentencePieceTrainer.train(input='cmd', model_prefix='cmd', model_type='bpe', vocab_size=500)

In [12]:
text_tokenizer = spm.SentencePieceProcessor(model_file='text.model')  ## YOUR CODE HERE ###
cmd_tokenizer = spm.SentencePieceProcessor(model_file='cmd.model') ## YOUR CODE HERE ###

**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [13]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 16

In [14]:
class TextToBashDataset(Dataset):
    ## YOUR CODE HERE ###
    def __init__(self, data_frame, text_tokenizer, cmd_tokenizer):
        self._df = data_frame.reset_index(drop=True)
        self._text_tokenizer = text_tokenizer
        self._cmd_tokenizer = cmd_tokenizer
        
    def __len__(self):
        return len(self._df)
    
    def __getitem__(self, idx):
        text = ' '.join(self._df['text_cleaned'][idx])
        cmd = self._df['cmd_cleaned'][idx]
        text = torch.LongTensor(self._text_tokenizer.Encode(text, add_bos=True, add_eos=True))
        cmd = torch.LongTensor(self._cmd_tokenizer.Encode(cmd, add_bos=True, add_eos=True))
        return text[:MAX_TEXT_LENGTH], cmd[:MAX_CODE_LENGTH]

In [15]:
train_ds = TextToBashDataset(train_data, text_tokenizer, cmd_tokenizer) ## YOUR CODE HERE ###
valid_ds = TextToBashDataset(valid_data, text_tokenizer, cmd_tokenizer) ## YOUR CODE HERE ###

In [16]:
def pad(sequence):
    max_length = max([el.shape[0] for el in sequence])
    padded_batch = []
    for el in sequence:
        padded_batch.append(torch.cat([el, torch.LongTensor([PAD_ID] * (max_length - el.shape[0]))]))
    return torch.stack(padded_batch)


def collate(batch):
    texts = [el[0] for el in batch]
    cmds = [el[1] for el in batch]
    return pad(texts), pad(cmds)


In [17]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=collate),
}

## Обучение бейзлайна (2 балла)

In [18]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [23]:
text_model_config = {
    'vocab': text_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

cmd_model_config = {
    'vocab': cmd_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

In [24]:
class TextToBashModel(nn.Module):
    def __init__(self, text_model_config, cmd_model_config):
        super(TextToBashModel, self).__init__()
        ## YOUR CODE HERE ##
        encoder_config = BertConfig(**text_model_config)
        cmd_model_config['is_decoder'] = True
        cmd_model_config['add_cross_attention'] = True
        decoder_config = BertConfig(**cmd_model_config)
        encoder_decoder_config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)
        config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)        
        self._coupling = EncoderDecoderModel(encoder_decoder_config)
        
    def forward(self, x):
        ## YOUR CODE HERE ##
        text_ids, cmd_ids = x
        mask = (text_ids != PAD_ID).float()
        return self._coupling(input_ids=text_ids, decoder_input_ids=cmd_ids[:, :-1], attention_mask=mask)        

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [25]:
## YOUR CODE HERE ##
from torch.utils.tensorboard import SummaryWriter
from  torch.nn.utils import clip_grad_norm_

class Trainer:

    def __init__(
            self, 
            model,
            optimizer, 
            pad_token_id,
            device,
            logdir='bert-text2bash',
            max_grad_norm=1
    ):
        self._device = device
        self._model = model.to(device)
        self._optimizer = optimizer
        self._pad_token_id = pad_token_id
        self._criterion = nn.CrossEntropyLoss()
        self._max_grad_norm = max_grad_norm
        
        self._writer = SummaryWriter(log_dir=logdir)
        self._step = 0
        self._n_epoch = 0
        

    def train(self, dataloaders, n_epochs):
        for epoch in range(n_epochs):
            self._train_step(dataloaders)
            self._n_epoch += 1
            
    def _validate(self, dataloader):
        total_loss = 0
        for texts, cmds in dataloader:
            texts = texts.to(self._device)
            cmds = cmds.to(self._device)
            with torch.no_grad():
                decoded = self._model((texts, cmds)).logits
            decoded = decoded.view(-1, decoded.shape[-1])
            cmds = cmds[:, 1:]
            cmds = cmds.reshape(-1)
            indicies = (cmds != PAD_ID)
            loss = self._criterion(decoded[indicies], cmds[indicies])
            total_loss += loss.item()
        return total_loss / len(dataloader)

    def _train_step(self, dataloaders):
        for i, (texts, cmds) in enumerate(dataloaders['train']):
            texts = texts.to(self._device)
            cmds = cmds.to(self._device)
            model_output = self._model((texts, cmds))
            decoded = model_output.logits
            decoded = decoded.reshape(-1, decoded.shape[-1])
            cmds = cmds[:, 1:]
            cmds = cmds.reshape(-1)
            indicies = (cmds != PAD_ID)
            loss = self._criterion(decoded[indicies], cmds[indicies])
            self._writer.add_scalar('Loss/train', loss.item(), self._step)
            loss.backward()
            clip_grad_norm_(self._model.parameters(), self._max_grad_norm)
            if i % 4 == 0:
                self._optimizer.step()
                self._optimizer.zero_grad()
            if i % 20 == 0:
                val_loss = self._validate(dataloaders['valid'])
                self._writer.add_scalar('Loss/valid', val_loss, self._step)
                print(f'epoch {self._n_epoch} iter {i} val_loss: {val_loss:.4}')
            self._step += 1

In [26]:
model = TextToBashModel(text_model_config, cmd_model_config)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
device = 'cuda'
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
trainer = Trainer(model, optimizer, PAD_ID, device)
trainer.train(loaders, 28)

epoch 0 iter 0 val_loss: 10.0
epoch 0 iter 20 val_loss: 9.328
epoch 0 iter 40 val_loss: 8.998
epoch 0 iter 60 val_loss: 8.748
epoch 0 iter 80 val_loss: 8.517
epoch 0 iter 100 val_loss: 8.274
epoch 0 iter 120 val_loss: 8.063
epoch 0 iter 140 val_loss: 7.864
epoch 0 iter 160 val_loss: 7.658
epoch 0 iter 180 val_loss: 7.458
epoch 0 iter 200 val_loss: 7.237
epoch 0 iter 220 val_loss: 7.029
epoch 0 iter 240 val_loss: 6.807
epoch 0 iter 260 val_loss: 6.606
epoch 0 iter 280 val_loss: 6.401
epoch 0 iter 300 val_loss: 6.196
epoch 0 iter 320 val_loss: 6.004
epoch 0 iter 340 val_loss: 5.811
epoch 0 iter 360 val_loss: 5.62
epoch 0 iter 380 val_loss: 5.432
epoch 0 iter 400 val_loss: 5.249
epoch 0 iter 420 val_loss: 5.078
epoch 0 iter 440 val_loss: 4.923
epoch 0 iter 460 val_loss: 4.764
epoch 0 iter 480 val_loss: 4.627
epoch 0 iter 500 val_loss: 4.485
epoch 0 iter 520 val_loss: 4.384
epoch 0 iter 540 val_loss: 4.262
epoch 0 iter 560 val_loss: 4.148
epoch 0 iter 580 val_loss: 4.046
epoch 0 iter 600 v

## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей. 

In [27]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device='cuda',
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        
        self.device = device
        
    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        ## YOUR CODE HERE ##
        if len(input_text_tokens.shape) == 1:
            input_text_tokens = input_text_tokens.unsqueeze(0)
        model = model.to(self.device)
        input_text_tokens = input_text_tokens.to(self.device)
        beam_output = model._coupling.generate(input_text_tokens,
                                               max_length=self.max_length,
                                               num_return_sequences=self.beam_width,
                                               output_scores=True,
                                               return_dict_in_generate=True,
                                               num_beams=self.beam_width,
                                               bos_token_id=self.bos_id,
                                               pad_token_id=self.pad_id,
                                               eos_token_id=self.eos_id,
                                               temperature=self.temperature)
        output = list(zip(beam_output.sequences.cpu().numpy(), beam_output.sequences_scores.cpu().numpy()))
        return [(seq[seq != self.pad_id], prob) for seq, prob in output]

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [28]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=0.6, device='cuda',
)

In [29]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = torch.concat(valid_ds[i], dim=0)
        pred = beam_search_engine.get_result(model, src)
        
        scores = []
        for x, proba in pred:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))


text: searches through the root filesystem ("/") for the file named chapter1, and prints the location
true: find / -name Chapter1 -type f -print
true cleaned: find Path -name Regex -type f -print
find Path -name Regex -type f -print -0.15314859
find Path -name Regex -type f -0.18424827
find Path -name Regex -type f -print0 -0.2090472
find Path -name Regex -prune -or -name Regex -print -0.25227
find Path -name Regex -type f -exec echo Regex {} \; -0.25723422
1.0

text: searches through the root filesystem ("/") for the file named chapter1.
true: find / -name Chapter1 -type f
true cleaned: find Path -name Regex -type f
find Path -name Regex -print -0.192784
find Path -name Regex -prune -or -name Regex -print -0.21505143
find Path -name Regex -type f -print -0.24424024
find Path -name Regex -or -name Regex -0.26890096
find Path -type f -name Regex -print -0.2783678
0.6666666666666666

text: searches through the root filesystem ("/") for the file named chapter1.
true: find / -name Chapter

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [30]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for i, (text, target_cmd) in enumerate(zip(df.text_cleaned.values, df.cmd.values)):
        input_tokens = torch.LongTensor(text_tokenizer.Encode(' '.join(text), add_bos=True, add_eos=True)) ## YOUR CODE HERE ##
        predictions = beam_engine.get_result(model, input_tokens)
        
        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)
        
        all_scores.append(max(object_scores))
    return all_scores

Ваша цель при помощи подбора параметров модели и генерации получить средний скор на валидации >= 0.2, скор `handcrafted` части теста >= 0.1. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

**Обратите внимание.** Так как датасет для обучения не очень большой, а данные достаточно нестабильные, подбор параметров может очень сильно влиять на модель. Некоторые полезные советы:
* Отслеживайте качество модели после каждой эпохи, не забывайте про early stopping
* Вы можете сразу приступить к следующей части. Побитие скора в этой части задания при помощи трюков из бонусной части считается валидным.

In [31]:
## YOUR CODE HERE ##
valid = np.mean(compute_all_scores(model, valid_data, beam_search_engine))
handcrafted = np.mean(compute_all_scores(model, test_data[test_data['origin'] == 'handcrafted'], beam_search_engine))
mined = np.mean(compute_all_scores(model, test_data[test_data['origin'] == 'mined'], beam_search_engine))
print(f'valid {valid}')
print(f'handcrafted {handcrafted}')
print(f'mined {mined}')

valid 0.19933333333333333
handcrafted 0.09190353143841516
mined -0.4139713374088374


## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на валидации >= 0.25, `mined` >= 0, `handrafted` >= 0.15.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...